# Deep Learning End Sem Q1
## Image classification on fashion mnist dataset

## Importing Libraries

In [24]:
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.utils import to_categorical

from keras.preprocessing.image import ImageDataGenerator

from keras.utils.vis_utils import plot_model

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Data Preparation

In [2]:
mnist_fashion=keras.datasets.fashion_mnist.load_data()
(train_images,train_labels),(test_images,test_labels)=mnist_fashion

In [8]:
label_names=['Tshirt/Top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankleboot']

def view_data(index):
    print('='*40)
    plt.imshow(train_images[index])
    plt.title('Label : ' + label_names[train_labels[index]])
    plt.show()

for index in range(3):
    view_data(index)

In [9]:
x_train=train_images.reshape(train_images.shape[0],28,28,1).astype('float32')/255
x_test=test_images.reshape(test_images.shape[0],28,28,1).astype('float32')/255

y_train=to_categorical(train_labels)
y_test=to_categorical(test_labels)

print('Training Shape :\nX -> {}\tY -> {}\n\nTesting Shape :\nX -> {}\tY -> {}'.format(x_train.shape,y_train.shape,x_test.shape,y_test.shape))

## Model

In [13]:
inputs=keras.Input(shape=(28,28,1),name='Input Layer')
x = layers.Conv2D(filters=64,kernel_size=3,activation='relu',name='Conv2D_Block_1_1')(inputs)
x = layers.Conv2D(filters=64,kernel_size=3,activation='relu',name='Conv2D_Block_1_2')(x)
x = layers.MaxPooling2D(pool_size=2,padding='same',name='Max_Pooling_Layer')(x)
x = layers.Dropout(0.5,name='Dropout_Layer_1')(x)
x = layers.Flatten()(x)
x = layers.Dense(128,activation='relu',name='Fully_Connected_1')(x)
x = layers.Dense(128,activation='relu',name='Fully_Connected_2')(x)
x = layers.Dense(128,activation='relu',name='Fully_Connected_3')(x)
x = layers.Dropout(0.5,name='Dropout_Layer_2')(x)
outputs=layers.Dense(10,activation='softmax',name='Output_Layer')(x)

baseline_model=keras.Model(inputs=inputs,outputs=outputs,name='BaselineModel')

In [14]:
plot_model(baseline_model,show_shapes=True,show_layer_names=True)

In [15]:
baseline_model.summary()

In [16]:
baseline_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [17]:
history=baseline_model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=50,
    steps_per_epoch=100
    verbose=1,
    validation_data=(x_test,y_test)
)

## Evaulation

In [21]:
def get_plot(trained_model,no_epochs):
    accuracy=trained_model.history['accuracy']
    loss=trained_model.history['loss']
    val_accuracy=trained_model.history['val_accuracy']
    val_loss=trained_model.history['val_loss']
    epochs=[i for i in range(1,no_epochs+1)]

    plt.figure(figsize=(12,8))
    plt.plot(epochs,accuracy,'r',label='Training')
    plt.plot(epochs,val_accuracy,'--',label='Validation')
    plt.legend()
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy',fontsize=15)
    plt.grid()
    plt.show()

    plt.figure(figsize=(12,8))
    plt.plot(epochs,loss,'r',label='Training')
    plt.plot(epochs,val_loss,'--',label='Validation')
    plt.legend()
    plt.title('Loss',fontsize=15)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid()
    plt.show()

In [22]:
get_plot(history,15)

In [46]:
prediction = baseline_model.predict(x_test)

pred = np.argmax(prediction, axis=1)

In [43]:
original = test_labels.astype('int')

cm = confusion_matrix(original, pred)

report = classification_report(original, pred, target_names=label_names)
print(report)